In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [2]:
training_exp_name = "training/01_03_24_WMH_Ensemble"

dfc = rs.load_configs(
    root / training_exp_name,
    properties=False,
)

  0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
dfc

,data,annotator,axis,in_channels,iters_per_epoch,num_slices,out_channels,preload,slicing,task,...,model,convs_per_block,filters,optim,lr,weight_decay,augmentations,epochs,eval_freq,path
0,ese.experiment.datasets.WMH,observer_o12,0,1,1000,1,2,True,dense,Amsterdam,...,ese.experiment.models.UNet,3,"(64, 64, 64, 64, 64)",torch.optim.Adam,0.0003,0.0,None,1000,5,/storage/vbutoi/scratch/ESE/training/01_03_24_...
1,ese.experiment.datasets.WMH,observer_o12,0,1,1000,1,2,True,dense,Amsterdam,...,ese.experiment.models.UNet,3,"(64, 64, 64, 64, 64)",torch.optim.Adam,0.0003,0.0,None,1000,5,/storage/vbutoi/scratch/ESE/training/01_03_24_...
2,ese.experiment.datasets.WMH,observer_o12,0,1,1000,1,2,True,dense,Amsterdam,...,ese.experiment.models.UNet,3,"(64, 64, 64, 64, 64)",torch.optim.Adam,0.0003,0.0,None,1000,5,/storage/vbutoi/scratch/ESE/training/01_03_24_...
3,ese.experiment.datasets.WMH,observer_o12,0,1,1000,1,2,True,dense,Amsterdam,...,ese.experiment.models.UNet,3,"(64, 64, 64, 64, 64)",torch.optim.Adam,0.0003,0.0,None,1000,5,/storage/vbutoi/scratch/ESE/training/01_03_24_...


In [4]:
%%yaml inference_config

log:
    root: '?'
    log_interval: 50
    track_image_level: True 
    track_pixel_level: True 
    ignore_index: '?' 

experiment:
    seed: 42

dataset:
    split: '?' 
    slicing: dense_full 
    input_type: volume 
    preload: True 

dataloader:
    batch_size: 1 
    num_workers: 1
    pin_memory: True 

calibration:
    conf_interval_start: 0.5
    conf_interval_end: 1.0
    num_bins: 10
    neighborhood_width: 3
    square_diff: False 

<IPython.core.display.Javascript object>

In [5]:
%%yaml model_cfg

model:
    exp_root : '?' 
    checkpoint: "max-val-dice_score"
    pretrained_select_metric: "val-dice_score"
    # ensemble: False 
    # ensemble_combine_fn: mean

<IPython.core.display.Javascript object>

In [6]:
%%yaml metrics_cfg 

qual_metrics:
    - Edge_ECE:
        func: ese.experiment.metrics.ece.edge_ece_loss
        metric_type: calibration
    - ECW_ECE:
        func: ese.experiment.metrics.ece.ecw_ece_loss
        metric_type: calibration

cal_metrics:
    - ECE:
        func: ese.experiment.metrics.ece.ece_loss
    - CW_ECE:
        func: ese.experiment.metrics.ece.cw_ece_loss
    - ELM:
        func: ese.experiment.metrics.elm.elm_loss
    - CW_ELM:
        func: ese.experiment.metrics.elm.cw_elm_loss

<IPython.core.display.Javascript object>

In [7]:
from ionpy.util import dict_product, Config
from ionpy.util.config import check_missing
from ese.scripts.utils import gather_exp_paths

# Need to define the experiment name
inference_exp_root = str(root / "inference/01_05_24_WMH_EnsembleInference")

# For ensembles, define the root dir.
ensemble_root = "/storage/vbutoi/scratch/ESE/training/01_03_24_WMH_Ensemble"
train_exp_paths = gather_exp_paths(ensemble_root)

# Get the inference options.
##################################################
dataset_options = {
    'log.root': [inference_exp_root],
    'model.exp_root': train_exp_paths,
    'log.ignore_index': [None],
    'dataset.split': ['cal']
}

base_cfg = Config(inference_config).update([model_cfg, metrics_cfg])

cfgs = []
for cfg_update in dict_product(dataset_options):
    new_cfg = base_cfg.update(cfg_update)
    check_missing(new_cfg) # Verify there are no ? in config.
    cfgs.append(new_cfg)

In [8]:
# from ese.experiment.analysis.inference import get_cal_stats
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# get_cal_stats(cfgs[0])

In [9]:
from ionpy.slite.submit import submit_jobs
from ese.experiment.analysis.inference import get_cal_stats

submit_jobs(
    exp_root=inference_exp_root,
    job_func=get_cal_stats,
    config_list=cfgs, 
    available_gpus=["0", "1", "2", "3"]
    )

Initalized SliteRunner
Submitted job id: 1224059.
Submitted job id: 1224068.
Submitted job id: 1224082.
Submitted job id: 1224088.
